In this file, the goal is to visualize the chamfer distance for all the regions

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from scipy.ndimage import distance_transform_edt
from numpy.lib.stride_tricks import sliding_window_view
from scipy.stats import ks_2samp

In [2]:
dir_path= "/neurospin/tmp/fred/models/2025-11-10"  

In [3]:
def edge_distance_kernel(mask, distance_threshold=5.0, sigma=2.0):
    """
    Compute a spatial weighting kernel emphasizing the center of a binary mask.
    Inside the mask:
      - Weights = 1 for pixels farther than `distance_threshold` from the edge.
      - Weights = Gaussian decay for pixels within `distance_threshold` of the edge.
    Outside the mask:
      - Weights = 0.
    Parameters
    ----------
    mask : np.ndarray
        2D or 3D binary mask defining the valid region (1 inside, 0 outside).
    distance_threshold : float
        Distance (in pixels/voxels) from the mask edge above which weight = 1.
    sigma : float
        Gaussian falloff width (controls how fast the weight decays near edges).
    Returns
    -------
    kernel : np.ndarray
        Weight map of same shape as `mask`, values in [0,1].
    """
    assert mask.ndim in (2, 3), "mask must be 2D or 3D"
    mask = (mask > 0).astype(np.uint8)
    # Compute distance to the nearest 0 (edge)
    dist_inside = distance_transform_edt(mask)
    # Gaussian decay near the edge
    kernel = np.ones_like(mask, dtype=float)
    near_edge = dist_inside < distance_threshold
    kernel[near_edge] = np.exp(-((distance_threshold - dist_inside[near_edge]) ** 2) / (2 * sigma ** 2))
    # Zero weight outside the mask
    kernel[mask == 0] = 0.0
    return kernel

In [4]:
def chamfer_sweep_weighted(
    binary,
    reconstruction,
    threshold=0.55,
    kernel=None,
    two_sided=True
):
    """
    Compute weighted Chamfer distance between a binary mask and thresholded
    reconstructions over a range of thresholds.
    Parameters
    ----------
    binary : np.ndarray
        2D or 3D binary ground truth (0/1).
    reconstruction : np.ndarray
        2D or 3D continuous reconstruction in [0,1].
    kernel : np.ndarray or None
        Weighting map of same shape as input. If None, use uniform weights = 1.
    two_sided : bool
        Whether to compute symmetric Chamfer (True) or one-sided (False).
    local_window : int or None
        If None, returns mean error per threshold (scalar).
        If int, compute the sum of weighted error in each valid window
        (no padding) and take the maximum as the score.
    """
    assert binary.shape == reconstruction.shape, "Shapes must match"
    assert binary.ndim in (2, 3), "Supports only 2D or 3D arrays"
    binary = (binary > 0.5).astype(np.uint8)
    if kernel is None:
        kernel = np.ones_like(binary, dtype=float)
    else:
        assert kernel.shape == binary.shape, "Kernel must have same shape as input"
    dist_bin = distance_transform_edt(1 - binary)


    recon_bin = (reconstruction > threshold).astype(np.uint8)
    dist_recon = distance_transform_edt(1 - recon_bin)
    if two_sided:
        error_map = binary * dist_recon + recon_bin * dist_bin
    else:
        error_map = recon_bin * dist_bin
    weighted_error = error_map * kernel

    chamfer_score = weighted_error.mean()

    return chamfer_score

In [5]:
def transform(inputs, outputs):

    return inputs[:,0,:,:,:], outputs[:,1,:,:,:]

In [6]:
def separate (input_epilepsy, output_epilepsy):
    input_control=input_epilepsy[0:19,:,:,:]
    output_control=output_epilepsy[0:19,:,:,:]


    return input_control, output_control

In [7]:
def transform_mask(mask, input):

    target_shape = input[0].shape

    pad_width = [(0, t - s) for s, t in zip(mask.shape, target_shape)]

    mask = np.pad(mask, pad_width, mode='constant', constant_values=0)
    return mask

In [8]:
mask_path = "/neurospin/dico/data/deep_folding/current/datasets/UkBioBank40/crops/2mm"

In [9]:
mask_path1= "/neurospin/dico/data/deep_folding/current/datasets"

In [10]:
import re

In [ ]:
selected_index = np.load("/neurospin/dico/fred/Runs/01_betaVAE/Program/2023_jlaval_STSbabies/betaVAE/notebooks/fred/PEPR_Marseille/All the subjects/histogram_non_zero/index_to_save.npy")

: 

In [ ]:
subfolders = sorted([
    f for f in os.listdir(dir_path)
    if os.path.isdir(os.path.join(dir_path, f))
])

rows, cols = 14, 4
total_plots = rows * cols

fig, axes = plt.subplots(rows, cols, figsize=(22, 32))
axes = axes.flatten()

# Table to  save the ks distance
ks_table =pd.DataFrame(columns=["regions", "UKB and PEPR_Marseille"])

for i, folder in enumerate(subfolders):
    print(i)
    regions=re.match(r"(.*?(left|right))", folder).group(1)
    ax = axes[i]
    current_path = os.path.join(dir_path, folder)

    input_UKB = os.path.join(current_path, "inputs.npy")
    output_UKB = os.path.join(current_path, "outputs.npy")

    input_hcp = os.path.join(current_path, "hcp", "inputs.npy")
    output_hcp = os.path.join(current_path, "hcp", "outputs.npy")

    input_epilepsy = os.path.join(current_path, "epilepsy_PBS", "inputs.npy")
    output_epilepsy = os.path.join(current_path, "epilepsy_PBS", "outputs.npy")

    input_PEPR = os.path.join(current_path, "PEPR_Marseille","inputs.npy")
    output_PEPR = os.path.join(current_path, "PEPR_Marseille","outputs.npy")

    input_UKB = np.load(input_UKB)
    output_UKB = np.load(output_UKB)
    input_UKB, output_UKB= transform(input_UKB, output_UKB)
    
    input_hcp = np.load(input_hcp)
    output_hcp = np.load(output_hcp)
    input_hcp, output_hcp= transform(input_hcp, output_hcp)

    input_epilepsy = np.load(input_epilepsy)
    output_epilepsy = np.load(output_epilepsy)
    input_epilepsy, output_epilepsy= transform(input_epilepsy, output_epilepsy)

    input_PEPR = np.load(input_PEPR)
    output_PEPR = np.load(output_PEPR)
    input_PEPR, output_PEPR = input_PEPR[selected_index], output_PEPR[selected_index]
    input_PEPR, output_PEPR= transform(input_PEPR, output_PEPR)

    input_control, output_control = separate(input_epilepsy, output_epilepsy)

    chamfer_UKB=[]
    chamfer_hcp=[]
    chamfer_control=[]
    chamfer_PEPR= []

    #find the folder in mask_path corresponding to folder
    subfolders_mask = [
    f for f in os.listdir(mask_path)
    if os.path.isdir(os.path.join(mask_path, f))
]
    
    for folder_mask in subfolders_mask:
        folder_clean = folder_mask.replace(".", "")  
        if folder.startswith(folder_clean):
            match = folder_mask
            break
    
    if "right" in folder:
        side_mask="Rmask.npy"
    elif "left" in folder:
        side_mask="Lmask.npy"

    mask=os.path.join(mask_path, match, "mask", side_mask)
    mask= np.load(mask)
    mask= mask[:,:,:,0]
    mask= transform_mask(mask,input_UKB)
    kernel=edge_distance_kernel(mask)

    for j in range(input_UKB.shape[0]):
        chamfer_UKB.append(chamfer_sweep_weighted(input_UKB[j,:,:,:], output_UKB[j,:,:,:], 0.55, kernel))

    for j in range(input_hcp.shape[0]):
        chamfer_hcp.append(chamfer_sweep_weighted(input_hcp[j,:,:,:], output_hcp[j,:,:,:], 0.55,kernel))

    for j in range(input_control.shape[0]):
        chamfer_control.append(chamfer_sweep_weighted(input_control[j,:,:,:], output_control[j,:,:,:],0.55, kernel))

    for j in range(input_PEPR.shape[0]):
        chamfer_PEPR.append(chamfer_sweep_weighted(input_PEPR[j,:,:,:], output_PEPR[j,:,:,:],0.55, kernel))

    ax.hist(chamfer_UKB, density=True, alpha=0.5, label="ukb", bins="auto")
    ax.hist(chamfer_hcp, density=True, alpha=0.5, label="hcp", bins="auto")
    ax.hist(chamfer_control, density=True, alpha=0.5, label="control in epilepsy_PBS", bins="auto")
    ax.hist(chamfer_PEPR,density=True, alpha=0.5, label="PEPR_Marseille", bins="auto")

    ax.set_title(regions, fontsize=8)
    ax.tick_params(axis='both', which='major', labelsize=6)
    if i == 0:
        ax.legend(fontsize=6)

    # KS distances

    ks_ukb_PEPR = ks_2samp(chamfer_UKB, chamfer_PEPR).statistic

    ks_table.loc[i] = [regions, ks_ukb_PEPR]

plt.tight_layout()
plt.show()

# Affichage final du tableau KS
print("\nTableau des distances de Kolmogorov smirnov :\n")
print(ks_table)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46


In [ ]:
names= ks_table.iloc[:,0]

value = ks_table.iloc[:, 1]

In [ ]:
visualize_distance= pd.DataFrame({ "region": names,
                                  "p": value})

In [ ]:
sorted_distance = visualize_distance.sort_values(by="p", ascending=False)

In [ ]:
sorted_distance.to_csv("Chamfer_Distance_UKB_PEPR_all_regions")